<a href="https://colab.research.google.com/github/maekawataiki/ambient-music-generation/blob/master/sample_colab_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ambient Music Generation
Ambient Music MIDI to MIDI generator

## Install Dependencies

In [ ]:
!git clone https://github.com/pachyderm/ambient-music-generation
!git clone https://github.com/jason9693/MusicTransformer-tensorflow2.0 music_transformer
%cd music_transformer
# !git clone https://github.com/jason9693/midi-neural-processor.git midi_processor
!git clone https://github.com/gudgud96/midi-neural-processor midi_processor
!pip install pretty_midi progress
!pwd

In [ ]:
from model import MusicTransformer, MusicTransformerDecoder
from custom.layers import *
from custom import callback
import params as par
from tensorflow.python.keras.optimizer_v2.adam import Adam
from data import Data
import utils
import datetime
import argparse
from midi_processor.processor import decode_midi, encode_midi
import pretty_midi
from google.colab import files

In [ ]:
beam = None
length = 2048
save_path = "../output.midi"

current_time = datetime.datetime.now().strftime('%Y%m%d-%H%M%S')
gen_log_dir = 'logs/mt_decoder/generate_'+current_time+'/generate'
gen_summary_writer = tf.summary.create_file_writer(gen_log_dir)

## Load Model

In [ ]:
with tf.device('/device:GPU:0'):
  model_path = "../ambient-music-generation/trained-models/ambient-musictransformer-model-1-may-27-2020_500-epochs"
  mt = MusicTransformerDecoder(loader_path=model_path)

## Import MIDI

In [ ]:
uploaded = files.upload()
file_names = list(uploaded.keys())

## Generate Result

In [ ]:
inputs = encode_midi(file_names[0])
with gen_summary_writer.as_default():
    result = mt.generate(inputs[:10], beam=beam, length=length, tf_board=True)

## Write MIDI

In [ ]:
decode_midi(result, file_path=save_path)

## Download MIDI

In [ ]:
files.download(save_path)